

====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Tim\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [133]:
import json, subprocess, re, pyodbc, os, time, random
from sqlalchemy import func, create_engine
from bs4 import BeautifulSoup 
import numpy as np # linear algebra 
from config import mongoConn, connstr
import nltk
from nltk.corpus import stopwords
sw=stopwords.words('english')
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime as dt
from datetime import date, timedelta

In [151]:
# executable_path = {'executable_path': ChromeDriverManager().install()}
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.google.com")



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Tim\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [116]:
with open('C:\\bom\\appsettings.json', encoding='utf-8-sig') as f:
    config = json.load(f)
pw = ''
for s in config['contexts']:
    if 'jira' in s['name']:
        pw=re.search('password,(.*);c:submit;', s['conn']).group(1)   



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Tim\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [173]:

def weeknum(dayname):
    if dayname == 'Monday':   return 0
    if dayname == 'Tuesday':  return 1
    if dayname == 'Wednesday':return 2
    if dayname == 'Thursday': return 3
    if dayname == 'Friday':   return 4
    if dayname == 'Saturday': return 5
    if dayname == 'Sunday':   return 6
def alldays(year, whichDayYouWant):
    d = date(year, 1, 1)
    d += timedelta(days = (weeknum(whichDayYouWant) - d.weekday()) % 7)
    while d.year == year:
        yield d
        d += timedelta(days = 7)

#for d in alldays(2021,'Monday'):
    #print(d)

In [184]:
# df = pd.read_csv(r'C:\Users\Tim\Documents\LOGS\jira.csv')  
def create_tasklog(df):
    dff=pd.DataFrame()
    df=df.copy()
    df['task'] = df['Issue key'] + ' ' + df['Summary'] 
    df['date'] = pd.to_datetime(df['Created'] )
    df['updated'] = pd.to_datetime(df['Updated'] )
    df=df[['task','date', 'updated']] 
    today = dt.datetime.today().strftime("%m-%d-%Y")  
    curyear = pd.to_datetime(today).year 
    yr = 2021
    dfs = []
    lst = []  
    for monday in alldays(yr,'Monday'):
        dy = pd.to_datetime(monday).day
        mo = pd.to_datetime(monday).month  
        for d in pd.date_range(start=monday, periods=5, freq='D'): 
            for t in range(900, 1800, 100):   
                if mo==12: 
                    dfd = df.loc[ (df.updated > pd.to_datetime(f'{yr}-{mo}-01')) & (df.updated < pd.to_datetime(f'{yr}-{mo}-30'))  ] 
                else:
                    dfd = df.loc[ (df.updated > pd.to_datetime(f'{yr}-{mo}-01')) & (df.updated < pd.to_datetime(f'{yr}-{mo+1}-01'))  ] 
                if len(dfd) > 1: 
                    idx = random.randrange(len(dfd)-1) 
                    lst.append({'date': re.search('(.*)\s.*'
                                , str(d)).group(1), 'time': str(t) 
                                , 'task': dfd.iloc[idx].task }) 
        if len(lst) > 1:  
            dfs.append({'date': str(monday), 'df': pd.DataFrame( lst )}) 
            pd.DataFrame(lst).to_csv(f'C:\\Users\\Tim\\Documents\\LOGS\\tasklog\\_{str(monday)}.csv', index=False)
            dff=pd.concat([dff,pd.DataFrame(lst)])
        lst = []
    return dff

In [171]:
import os 
directory = r'C:\Users\Tim\Downloads'
for entry in os.scandir(directory):
    if ("JIRA" in entry.path and entry.path.endswith("csv")) and entry.is_file(): 
        os.remove(entry.path)  
driver = webdriver.Chrome(ChromeDriverManager().install()) 
driver.implicitly_wait(5)
driver.get(f"https://dayman.cyber-balance.com/jira/login.jsp")
driver.find_element_by_css_selector('#login-form-username').send_keys('tim.kopp')
driver.find_element_by_css_selector('#login-form-password').send_keys(pw)
driver.find_element_by_css_selector('#login-form-submit').click()
driver.get(f"https://dayman.cyber-balance.com/jira/issues/?jql=assignee%20%3D%20currentUser()%20AND%20project%3DCyberScope")
driver.implicitly_wait(5)
driver.find_element_by_css_selector('.header-operations').click()
driver.find_element_by_css_selector('#currentCsvFields').click()
driver.find_element_by_css_selector('#csv-export-dialog-export-button').click() 
# html=driver.page_source
# driver.quit()
# df = pd.read_html( html )[0]   
# df



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Tim\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [172]:
df=pd.DataFrame();
for entry in os.scandir(directory):
    if ("JIRA" in entry.path and entry.path.endswith("csv")) and entry.is_file(): 
        print(entry.path)
        df=pd.read_csv(entry.path)   
dff = create_tasklog(df)
dff

C:\Users\Tim\Downloads\CyberBalance JIRA 2021-08-23T15_40_21+0000.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554 entries, 0 to 553
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Issue Type           554 non-null    object
 1   Issue key            554 non-null    object
 2   Issue id             554 non-null    int64 
 3   Summary              554 non-null    object
 4   Assignee             554 non-null    object
 5   Reporter             554 non-null    object
 6   Priority             554 non-null    object
 7   Status               554 non-null    object
 8   Resolution           512 non-null    object
 9   Created              554 non-null    object
 10  Updated              554 non-null    object
 11  Due Date             1 non-null      object
 12  Component/s          32 non-null     object
 13  Project key          554 non-null    object
 14  Project name         554 non-null   

In [187]:
dff 

,date,time,task
0,2021-01-04,900,CS-7845 Adjust Data Type and Validation for Me...
1,2021-01-04,1000,CS-7863 FY21 Risk Management Assessment Q1: Va...
2,2021-01-04,1100,"CS-7899 BOD 18-02 POA&M: Update Assessment ""Ty..."
3,2021-01-04,1200,CS-7882 Adjust BOD 18-02 POA&M Submission Stat...
4,2021-01-04,1300,CS-7880 BOD 18-02 POA&M Pilot Feedback: Sectio...
...,...,...,...
40,2021-09-03,1300,CS-8433 FY21 Annual SAOP Data Call: Section 7 ...
41,2021-09-03,1400,CS-8428 FY21 Annual SAOP Data Call: Section 2 ...
42,2021-09-03,1500,CS-8422 FY21 Annual SAOP Data Call: Section 10...
43,2021-09-03,1600,CS-8419 FY21 Annual SAOP Data Call: Section 7 ...


In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install()) 
driver.implicitly_wait(5)
driver.get(f"https://dayman.cyber-balance.com/jira/login.jsp") 
driver.find_element_by_css_selector('#login-form-submit').click()
driver.get(f"https://dayman.cyber-balance.com/jira/issues/?jql=assignee%20%3D%20currentUser()%20AND%20project%3DCyberScope")
driver.implicitly_wait(5)
driver.find_element_by_css_selector('.header-operations').click()
driver.find_element_by_css_selector('#currentCsvFields').click()
driver.find_element_by_css_selector('#csv-export-dialog-export-button').click() 